In [ ]:
# Loading the credentials from the env file
from gen_ai_hub.proxy.gen_ai_hub_proxy import GenAIHubProxyClient
from dotenv import load_dotenv
import os

load_dotenv(override=True)

# Fetching environment variables
AICORE_BASE_URL = os.getenv("AICORE_BASE_URL")
AICORE_RESOURCE_GROUP = os.getenv("AICORE_RESOURCE_GROUP")
AICORE_AUTH_URL = os.getenv("AICORE_AUTH_URL")
AICORE_CLIENT_ID = os.getenv("AICORE_CLIENT_ID")
AICORE_CLIENT_SECRET = os.getenv("AICORE_CLIENT_SECRET")

# Initializing the GenAIHubProxyClient
client = GenAIHubProxyClient(
    base_url=AICORE_BASE_URL,
    auth_url=AICORE_AUTH_URL,
    client_id=AICORE_CLIENT_ID,
    client_secret=AICORE_CLIENT_SECRET,
    resource_group=AICORE_RESOURCE_GROUP
)


# Dependencies and Helper Functions

In [ ]:
!pip install rich PyYAML "sap-ai-sdk-gen[all]"

In [ ]:
from gen_ai_hub.proxy import get_proxy_client
import pathlib
import yaml

from ai_api_client_sdk.models.input_artifact_binding import InputArtifactBinding
from ai_api_client_sdk.models.parameter_binding import ParameterBinding
from ai_api_client_sdk.models.artifact import Artifact
from ai_api_client_sdk.models.label import Label

SUPPORTED_MODELS = [
    'gemini-2.5-pro:001',
    'gpt-4o:2024-08-06'
]

SUPPORTED_METRICS = ["LLMaaJ:Sem_Sim_1", "JSON_Match"]

In [4]:
client = get_proxy_client()

In [6]:
from logging import PlaceHolder
from pydantic import BaseModel
from typing import List
import re
import requests
import json

class PromptTemplate(BaseModel):
    role: str
    content: str


class PromptTemplateSpec(BaseModel):
    template: List[PromptTemplate]


    @property
    def placeholders(self):
        placeholders = set()
        pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
        for message in self.template:
            placeholders.update(pattern.findall(message.content))
        return placeholders

    @classmethod
    def from_optimizer_result(cls, input_):
        placeholders = input_["user_message_template_fields"]
        def replace(msg):
            for key in placeholders:
                msg = msg.replace("{"+key+"}", "{{?"+ key + "}}")
            return msg

        return cls(
            template=[
                {
                    "role": "system",
                    "content": replace(input_["system_prompt"]),
                },{
                    "role": "user",
                    "content": replace(input_["user_message_template"]),
                }
            ]
        )

    def escape_curly_brackets(self) -> "PromptTemplateSpec":
        # 1. Hide each {{?key}} placeholder with a unique token
        placeholder_pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
        mapping = {}
        counter = 1

        def _hide(match):
            nonlocal counter
            token = f"__PLACEHOLDER_{counter}__"
            mapping[token] = match.group(0)
            counter += 1
            return token

        new_templates = []
        for msg in self.template:
            # a) hide custom placeholders
            hidden = placeholder_pattern.sub(_hide, msg.content)
            # b) escape all remaining braces
            escaped = hidden.replace('{', '{{').replace('}', '}}')
            # c) restore the original placeholders
            print(mapping)
            for token, original in mapping.items():
                escaped = escaped.replace(token, original)

            new_templates.append(PromptTemplate(role=msg.role, content=escaped))

        # return a fresh copy
        return PromptTemplateSpec(template=new_templates)



def fetch_prompt_template(prompt_template: str) -> PromptTemplateSpec:
    headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    url = f"{client.ai_core_client.base_url}/lm/promptTemplates"
    scenario, sep, name = prompt_template.partition("/")
    if sep:
        name, sep, version = name.partition(":")
    if sep:
        body = {"name": name,
                "version": version,
                "scenario": scenario,
                "includeSpec": True
            }
        response =  requests.get(url, headers=headers, params=body)
        response.raise_for_status()
        response = response.json()
        if response["count"] > 0:
            response = response["resources"][0]
        else:
            raise ValueError(f"Prompt template {name} not found.")
    else:
        url += f"/{prompt_template}"
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        response = response.json()
    return PromptTemplateSpec.model_validate(response["spec"])

def load_prompt_template(prompt: str | pathlib.Path | list | dict | PromptTemplateSpec) -> PromptTemplateSpec:
    if isinstance(prompt, PromptTemplateSpec):
        return prompt
    if isinstance(prompt, (str, pathlib.Path)) and pathlib.Path(prompt).exists():
        with open(prompt, "r") as f:
            prompt = yaml.safe_load(f)
    elif isinstance(prompt, str):
        return fetch_prompt_template(prompt)
    if isinstance(prompt, dict):
        # expect dict with keys "system" [optional] and "user"
        messages = []
        if "system" in prompt:
            messages.append({"role": "system", "content": prompt["system"]})
        messages.append({"role": "user", "content": prompt["user"]})
        return PromptTemplateSpec(template=messages)
    elif isinstance(prompt, list):
        # expect list of dicts with keys "role" and "content"
        return PromptTemplateSpec(template=messages)
    else:
        raise ValueError("Prompt must be a string, Path, list or dict")


def push_prompt_template(prompt_template: PromptTemplateSpec,
                         prompt_template_name_registry: str,
                         prompt_template_version: str,
                         scenario: str,
                         update=False):
    headers = {
        **client.request_header,
        "Content-Type": "application/json",
    }
    url = f"{client.ai_core_client.base_url}/lm/promptTemplates"
    body = {"name": prompt_template_name_registry,
            "version": prompt_template_version,
            "scenario": scenario}
    res = requests.get(url, headers=headers, params=body).json()
    if res["count"] > 0 and not update:
        print(f"Prompt template {prompt_template_name_registry} already exists. Use update=True to update.")
        return res["resources"][0]
    # Prepare body

    body["spec"] = prompt_template.model_dump()
    # Prepare headers
    response = requests.post(url, headers=headers, json=body)
    # Handle response
    if response.status_code == 201:
        response = response.json()
    elif response.status_code in (400, 409, 413):
        # Return error details
        raise requests.HTTPError(f"Upload failed ({response.status_code}): {response.text}")
    else:
        response.raise_for_status()
    return response.json()


import re

def convert_py_notation(template):
    pattern = re.compile(r'\{\{\s*\?\s*(\w+)\s*\}\}')
    return pattern.sub(lambda match: "{" + match.group(1) + "}", template)


def validate_prompt(prompt: PromptTemplateSpec):
    values = {k: "???" for k in prompt.placeholders}

    for message in prompt.template:
        if message.role == "user":
            try:
                convert_py_notation(message.content).format(**values)
            except KeyError as err:
                msg = ["Unexpected key error when running test formatting."]
                msg += ["This is most likeyly due to unescaped curly brackets."]
                msg += ["You can try fixing this by running `prompt = prompt.escape_curly_brackets()` and use the new prompt template."]
                raise ValueError("\n".join(msg)) from err
    return True




from rich.console import Console
from rich.highlighter import RegexHighlighter
from rich.theme import Theme
from rich.panel import Panel
from rich import print

class TemplateHighlighter(RegexHighlighter):
    """Apply style to anything that looks like an email."""

    base_style = "template."
    highlights = [r"(?P<placeholder>\{\{\s*\?[^\{\}\s]+\s*\}\})"]

highlighter = TemplateHighlighter()
theme = Theme({"template.placeholder": "bold magenta", "example.email": "bold magenta"})
console = Console(highlighter=highlighter, theme=theme)


def print_prompt_template(prompt_template: PromptTemplateSpec | str | pathlib.Path, addition: str | None = None):

    prompt_template = load_prompt_template(prompt_template)
    addition = f' - {addition}' if addition else ''

    for message in prompt_template.template:
        if message.role == "system":
            console.print(Panel(highlighter(message.content), title="System Message" + addition, border_style="red"))
        elif message.role == "user":
            console.print(Panel(highlighter(message.content), title="User Message" + addition, border_style="green"))
        else:
            console.print(Panel(highlighter(message.content), title="Assistant Message" + addition))



In [7]:
from typing import List
import requests
import mimetypes
from urllib.parse import quote
import pathlib
import json


def validate_dataset(dataset: str | pathlib.Path | list, expected_keys: None | List[str] = None) -> bool:
    if isinstance(dataset, (str, pathlib.Path)):
        with open(dataset, "r") as f:
            try:
                dataset = json.load(f)
            except json.JSONDecodeError as e:
                raise ValueError(f"Invalid JSON in file: {e}")
    if not isinstance(dataset, list):
        raise ValueError("Dataset must be a list of dictionaries.")

    def validate_item(item: dict, excepted_keys: None | List[str]) -> bool:
        excepted_keys = set(excepted_keys) if excepted_keys else None
        if set(item.keys()) != {"fields", "answer"}:
            raise ValueError("Each item must contain 'fields' and 'answer' keys.")
        if not isinstance(item["fields"], dict):
            raise ValueError("'fields' must be a dictionary.")
        fields = set(item["fields"].keys())
        if excepted_keys is not None:
            if fields != excepted_keys:
                if fields.difference(excepted_keys):
                    raise ValueError(f"Unexpected keys in 'fields'. Expected: {excepted_keys}, Found: {fields}")
                if excepted_keys.difference(fields):
                    raise ValueError(f"Missing keys in 'fields'. Expected: {excepted_keys}, Found: {fields}")
        if not all([isinstance(k, str) for k in item["fields"].values()]):
            raise ValueError("All values in 'fields' must be strings.")
        return fields

    excepted_keys = expected_keys
    for i, item in enumerate(dataset):
        if not isinstance(item, dict):
            raise ValueError("Each item in the dataset must be a dictionary.")
        try:
            excepted_keys = validate_item(item, excepted_keys)
        except ValueError as e:
            raise ValueError(f"Error in entry {i}") from e
    return True


def upload_dataset(secret: str,
                   local_path: str | pathlib.Path,
                   remote_path: str,
                   scenario: str,
                   description: str | None = None,
                   overwrite: bool = False,
                   expected_keys: None | List[str] = None,

                   allow_bucket_root: bool = False) -> str:
    # Validate dataset
    validate_dataset(local_path, expected_keys)
    # check if secret exists
    secrets = [r.name for r in client.ai_core_client.object_store_secrets.query().resources]
    if secret not in secrets:
        raise ValueError(f"Secret '{secret}' not found in object store secrets. Known secrets: {secrets}")

    # Check if local path exists
    remote_path = remote_path.lstrip("/")
    if "/" not in remote_path and not allow_bucket_root:
        raise ValueError(
            "Remote path must use subdirectories. Otherwise the whole bucket will be used as an input artifact. Set allow_bucket_root=True to allow this."
        )

    # URL-encode the path parameter
    path = f"{secret}/" + remote_path.lstrip("/")
    encoded_path = quote(path, safe="")
    url = f"{client.ai_core_client.base_url}/lm/dataset/files/{encoded_path}"
    params = {"overwrite": str(overwrite).lower()}

    # Prepare headers
    headers = {
        **client.request_header,
        "Content-Type": "application/octet-stream",
    }
    # Guess MIME type
    guessed_type, _ = mimetypes.guess_type(local_path)
    if guessed_type:
        headers["Content-Type"] = guessed_type

    with open(local_path, "rb") as f:
        response = requests.put(url, params=params, headers=headers, data=f)

    # Handle response
    if response.status_code == 201:
        response = response.json()
    elif response.status_code in (400, 409, 413):
        # Return error details
        raise requests.HTTPError(f"Upload failed ({response.status_code}): {response.text}")
    else:
        response.raise_for_status()
    artifact_url = "/".join(response["url"].split("/")[:-1])
    for artifact in client.ai_core_client.artifact.query().resources:
        if response["url"].startswith(artifact.url + "/"):
            return artifact, response["url"].removeprefix(artifact.url).lstrip("/")

    # Create new artifact
    path = response["url"].split("/")[-1]
    new_artifact = client.ai_core_client.artifact.create(
        name=f"{scenario}-prompt-optimization",
        kind=Artifact.Kind.DATASET,
        url=artifact_url,
        scenario_id=scenario,
        description="Datasets for prompt optimization" if description is None else description,
        resource_group=headers[client.ai_core_client.rest_client.resource_group_header]
    )
    return new_artifact, path



## Create Config

In [ ]:
old_new_name_mapping = {
    "gemini-2.5-pro:001": "gemini-2.5-pro:001",
    "gpt-4o:2024-08-06": "openai/gpt-4o-2024-08-06"
}

old_new_name_mapping.update({old_new_name_mapping[k]: k for k, v in old_new_name_mapping.items()})


def create_config(metric: str,
                  reference_model: str,
                  targets: dict,
                  dataset_path: str,
                  scenario: str,
                  prompt: PromptTemplateSpec) -> str:
    assert metric in SUPPORTED_METRICS, f"Unsupported metric: {metric}. Supported metrics: {SUPPORTED_METRICS}"
    assert reference_model in SUPPORTED_MODELS, f"Unsupported reference model: {reference_model}. Supported models: {SUPPORTED_MODELS}"
    assert all(model in SUPPORTED_MODELS for model in targets.keys()), f"Unsupported target models: {targets}. Supported models: {SUPPORTED_MODELS}"
    input_parameters = [
        ParameterBinding(key="dataset", value=dataset_path),
        ParameterBinding(key="optimizationMetric", value=metric),
        ParameterBinding(key="basePrompt", value=f'{scenario}/{prompt["name"]}:{prompt["version"]}'),
        ParameterBinding(key="baseModel", value=reference_model),
        ParameterBinding(key="targetModels", value=','.join(targets.keys())),
        ParameterBinding(key="targetPromptMapping", value=",".join([f"{old_new_name_mapping[k]}={v}" for k, v in targets.items()]))
        
        
    ]
    existing_configs = client.ai_core_client.configuration.query(scenario_id='genai-optimizations', executable_ids=['genai-optimizations'])
    params = {par.key: par.value for par in input_parameters}
    for conf in existing_configs.resources:
        if {par.key: par.value for par in conf.parameter_bindings} == params:
            return conf.id
    
    input_artifacts = [InputArtifactBinding(key="prompt-data", artifact_id=artifact_id)]

    response = client.ai_core_client.configuration.create(
        name = "prompt-optimization-config", # custom name of configuration
        scenario_id = "genai-optimizations", # value from workflow
        executable_id = "genai-optimizations", # value from workflow
        resource_group = resource_group,
        parameter_bindings = input_parameters,
        input_artifact_bindings = input_artifacts
    )

    return response.id



In [9]:
from rich.console import Console
from rich.table import Table

def fetch_results(execution_id):
    response = client.ai_core_client.execution.get(execution_id = execution_id)
    if response.status.name not in {'DEAD', 'COMPLETED'}:
        raise RuntimeError('Execution not finished!')
    path = f"default/{execution_id}/result-data/results.json"
    encoded_path = quote(path, safe="")
    url = f"{client.ai_core_client.base_url}/lm/dataset/files/{encoded_path}"
    headers = {
        **client.request_header,
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()# results = response.json()


def print_result(result):
    origin_model = result["origin_model"]
    table = Table(title="Performance")
    table.add_column("Model", justify="right", style="cyan", no_wrap=True)
    table.add_column("Pre Optimization", style="magenta")
    table.add_column("Post Optimization", justify="right", style="green")
    table.add_row(origin_model["model_name"], f'{origin_model["score"]:.3f}', "n/a - reference run")
    for m in result["target_models"]:
        table.add_row(m["model_name"], f'{m["pre_optimization_score"]:.3f}', f'{m["post_optimization_score"]:.3f}')
    console.print(table)
    for m in result["target_models"]:
        print_prompt_template(PromptTemplateSpec.from_optimizer_result(m), addition=m['model_name'])


### Download Demo Data

In [ ]:
import pathlib

files = [
    ("default/example/base-prompt.yaml", "./facility_prompt.yaml"),
    ("default/example/facility-train.json", "./facility-train.json")
]

for remote, local in files:
    local_path = pathlib.Path(local)
    if not local_path.exists():
        url = f"{client.ai_core_client.base_url}/lm/dataset/files/{remote}"
        headers = {
            **client.request_header,
        }
        response = requests.get(url, headers=headers)
        with local_path.open("w") as stream:
            stream.write(response.text)

In [9]:
resource_group = client.request_header[client.ai_core_client.rest_client.resource_group_header]

# Start Prompt Optimizer Run

### Loading a Local Prompt Template

**The prompt template is structured in a `system` and a `user` message. Placeholders in the prompt template have to be wrapped in `{{?key}}`.**


Your prompt can be provided in any of the following forms and will be normalized to a `PromptTemplateSpec` under the hood:

#### From Local Disk
**A file path** (`str` or `Path`) pointing to a YAML or JSON file defining either:
  - a **mapping** with keys  
    - `"user"` (required) and  
    - `"system"` (optional)

```yaml
system: |-
    You are a helpful assistant
assistant: |-
    Write a poen on {{?topic}}
```
or  
  - a **list** of message objects, each with `"role"` (e.g. `"system"` or `"user"`) and `"content"` (string)

```yaml
- role: system
  content: |-
    You are a helpful assistant
- role: user
  content: |-
    Write a poen on {{?topic}}
```


#### Alternative: Prompt Registry
- **A lookup string** of the form `"<scenario>/<name>:<version>"` (or just `"<name>:<version>"`) will be fetched from the AI-core prompt‐template API; if you omit the version, the latest will be returned.  


In [11]:
base_prompt_template = "./facility_prompt.yaml" # local path to the prompt template or Prompt Repository identifier


prompt = load_prompt_template(base_prompt_template) # .escape_curly_brackets() if validation fails.
print_prompt_template(prompt)
print(f"Prompt template loaded successfully. Placeholders found are: {prompt.placeholders}")
assert validate_prompt(prompt)


╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ You are a helpful assistant                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── User Message ──────────────────────────────────────────────────╮
│ Giving the following message:                                                                                   │
│ ---                                                                                                             │
│ {{?input}}                                                                                                      │
│ ---                                                                                                             │
│ Extract and return a json with the follwoing keys and values:                                                   │
│ - "urgency" as one of `high`, `medium`, `low`                                                                   │
│ - "sentiment" as one of `negative`, `neutral`, `positive`                                                       │
│ - "categories" Create a dictionary with categories as keys and boolean values (True/False), where the value     │
│ indicates whether the category is one of the best matching support category tags from:                          │
│ `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`,                     │
│ `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`,             │
│ `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`,            │
│ `facility_management_issues`                                                                                    │
│ Your complete message should be a valid json string that can be read directly and only contain the keys         │
│ mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Prompt template loaded successfully. Placeholders found are: {'input'}

Check if all expected placeholders were found.

### Validating Local Dataset

Your dataset must be a JSON‐serializable list where each element is a dictionary with exactly two keys: **`fields`** and **`answer`**. The **`fields`** value should itself be a dictionary whose keys (e.g. `"question"`, `"hint"`, `"term"`, etc.) are **identical** across every entry and whose values are all strings. The **`answer`** value must also be a string.


You can validate your dataset using the `validate_dataset` method.

If validation is not passed succesfully this are might be the reasons:


| Condition                              | Exception Raised (inner)                                           | Outer Message                    |
| -------------------------------------- | ------------------------------------------------------------------ | -------------------------------- |
| Non-list top-level                     | N/A                                                                | `Dataset must be a list…`        |
| Item not a dict                        | N/A                                                                | `Each item…must be a dictionary` |
| Wrong item keys                        | `ValueError("Each item must contain 'fields' and 'answer' keys.")` | `Error in entry i`               |
| `"fields"` not a dict                  | `ValueError("'fields' must be a dictionary.")`                     | `Error in entry i`               |
| Field name mismatch (extra or missing) | `ValueError("Unexpected keys…")` or `ValueError("Missing keys…")`  | `Error in entry i`               |
| Non-string field value                 | `ValueError("All values in 'fields' must be strings.")`            | `Error in entry i`               |
| Invalid JSON file                      | `ValueError("Invalid JSON in file:…")`                             | N/A                              |


In [28]:
dataset_local_path="./facility-synth-train/facility-train.json" # local path to the dataset

assert validate_dataset(dataset_local_path), "Dataset not valid"

### Remaining Config parameter

In [29]:
scenario = "genai-optimizations"

base_prompt_template_registry = "evaluate-base:0.0.1"  # name:version for the template in the registry

dataset_secret="default" # secret name in the object store you want to use to store the dataset
dataset_remote_path="datasets/facility-train.json" # remote path in the object store to store the dataset

reference_model = "gpt-4o:2024-08-06"
# Dictionary of models to optimize with their corresponding prompt template names under which the optimized prompt should be stored in the registry
targets = {
    "gemini-2.5-pro:001": "evaluate-base-gemini-2_5-pro:0.0.1"
}

# Metric to use for optimization
metric = "JSON_Match"


## Push Local Prompt to Registry

In [30]:
base_template = load_prompt_template(base_prompt_template)
prompt_template_name_registry, _, prompt_template_version = base_prompt_template_registry.partition(":")
prompt = push_prompt_template(prompt_template=base_template,
                              prompt_template_name_registry=prompt_template_name_registry,
                              prompt_template_version=prompt_template_version,
                              scenario=scenario,
                              update=False
)

print(f"Prompt present in registry under id {prompt['id']}")

print('\n\n=== Base Prompt ===')
print_prompt_template(prompt["id"])

Prompt template evaluate-base already exists. Use update=True to update.

Prompt present in registry under id 3a9cfc20-f972-4720-8d0e-3ac48f77f391

=== Base Prompt ===

╭──────────────────────────────────────────────── System Message ─────────────────────────────────────────────────╮
│ You are a helpful assistant                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── User Message ──────────────────────────────────────────────────╮
│ Giving the following message:                                                                                   │
│ ---                                                                                                             │
│ {{?input}}                                                                                                      │
│ ---                                                                                                             │
│ Extract and return a json with the follwoing keys and values:                                                   │
│ - "urgency" as one of `high`, `medium`, `low`                                                                   │
│ - "sentiment" as one of `negative`, `neutral`, `positive`                                                       │
│ - "categories" Create a dictionary with categories as keys and boolean values (True/False), where the value     │
│ indicates whether the category is one of the best matching support category tags from:                          │
│ `emergency_repair_services`, `routine_maintenance_requests`, `quality_and_safety_concerns`,                     │
│ `specialized_cleaning_services`, `general_inquiries`, `sustainability_and_environmental_practices`,             │
│ `training_and_support_requests`, `cleaning_services_scheduling`, `customer_feedback_and_complaints`,            │
│ `facility_management_issues`                                                                                    │
│ Your complete message should be a valid json string that can be read directly and only contain the keys         │
│ mentioned in the list above. Never enclose it in ```json...```, no newlines, no unnessacary whitespaces.        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Push Local Dataset to Object Store and Create Artifact

In [ ]:
artifact, dataset_path = upload_dataset(
    secret=dataset_secret,
    local_path=dataset_local_path,
    remote_path=dataset_remote_path,
    expected_keys=base_template.placeholders,
    scenario=scenario,
    overwrite=True,
    allow_bucket_root=True
)

print(f"Dataset uploaded to {artifact.url}/{dataset_path} -> Artifact ID: {artifact.id}")


## Create Prompt Optimizer Config

In [ ]:
configuration_id = create_config(metric=metric,
                                 reference_model=reference_model,
                                 targets=targets,
                                 dataset_path=dataset_path,
                                 scenario=scenario,
                                 prompt=prompt
                            )


## Start Prompt Optimizer

In [ ]:
response = client.ai_core_client.execution.create(
    configuration_id = configuration_id, # Change this value.
    resource_group = "default"
)

execution_id = response.id
print('Execution started with ID:', execution_id)

## Check Execution Status

In [ ]:
result = fetch_results(execution_id)
print_result(result)